## 문제 설명

Fasion-MNIST 데이터셋은 10개의 의류 카테고리로 구성된 흑백 이미지 데이터셋입니다. 본 예선 문제에서는 아래 두 클래스를 구분하는 **이진 분류기**를 양자 알고리즘을 활용하여 설계하는 것이 목표입니다. 아래의 이미지에서 T-shirt/top과 Shirt 라벨에 대해서만 분류하는 것입니다. 만들어야 할 모델은 양자 회로(quantum circuit)를 기반으로 하며, 잡음 없는 양자 시뮬레이터에서 실행가능해야 합니다.
- 클래스 0 : **T-shirt/top**
- 클래스 6 : **Shirt**

## 제약 조건
본 예선에서는 양자 모델 설계에 대한 현실적인 자원 제약을 고려하며, 다음과 같은 기술적 제한을 준수하여야 합니다.
- 양자 개발 프레임워크는 PennyLane을 사용합니다.
- 참가자는 최대 8 큐빗까지 사용 가능합니다.
- 사용되는 큐빗 수는 모델 전반에 걸쳐 유지되어야 하며, ancilla qubit 또는 mid-circuit measurement는 허용되지 않습니다.
- 전체 회로의 깊이(depth)는 최대 30으로 제한합니다.
- 모델 내 학습 가능한 퀀텀 레이어 파라미터(num_trainable_params)의 총 개수는 8개 이상 ~ 60개 이하로 제한합니다.
- 입력 데이터를 양자 상태로 인코딩하는 과정은 Amplitude Encoding, Angle Encoding, IQP-style Embedding 등을 자유롭게 활용할 수 있습니다.
- 참가자의 판단에 따라 데이터 차원 축소가 필요한 경우, 비지도 차원 축소 기법(PCA 등)의 사용을 허가합니다.
- 고전 머신러닝·딥러닝 모델과의 하이브리드 구성을 허용하되, 양자·고전 파라미터를 모두 합한 총 학습 가능 파라미터 수는 50,000(50K)개 이하로 제한합니다.

In [1]:
# 필요한 라이브러리 import
import pennylane as qml
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Fashion-MNIST 데이터 로드 및 전처리
def load_fashion_mnist_binary():
    # Fashion-MNIST 데이터셋 로드
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
    
    # T-shirt/top (0)과 Shirt (6) 클래스만 필터링
    def filter_classes(dataset, target_classes=[0, 6]):
        indices = [i for i, (_, label) in enumerate(dataset) if label in target_classes]
        data = torch.stack([dataset[i][0] for i in indices])
        labels = torch.tensor([dataset[i][1] for i in indices])
        # 라벨을 0, 1로 변경 (0: T-shirt/top, 1: Shirt)
        labels = (labels == 6).long()
        return data, labels
    
    train_data, train_labels = filter_classes(train_dataset)
    test_data, test_labels = filter_classes(test_dataset)
    
    return train_data, train_labels, test_data, test_labels

# 데이터 로드
train_data, train_labels, test_data, test_labels = load_fashion_mnist_binary()
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Train labels distribution: {torch.bincount(train_labels)}")
print(f"Test labels distribution: {torch.bincount(test_labels)}")

Train data shape: torch.Size([12000, 1, 28, 28])
Test data shape: torch.Size([2000, 1, 28, 28])
Train labels distribution: tensor([6000, 6000])
Test labels distribution: tensor([1000, 1000])


In [3]:
# 데이터 차원 축소 (PCA 사용)
def preprocess_data(train_data, test_data, n_components=8):
    # 데이터를 평면화
    train_flat = train_data.view(train_data.shape[0], -1).numpy()
    test_flat = test_data.view(test_data.shape[0], -1).numpy()
    
    # 표준화
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_flat)
    test_scaled = scaler.transform(test_flat)
    
    # PCA로 차원 축소 (8차원으로 축소하여 8 큐빗에 맞춤)
    pca = PCA(n_components=n_components)
    train_pca = pca.fit_transform(train_scaled)
    test_pca = pca.transform(test_scaled)
    
    # 정규화 [-π, π] 범위로
    train_normalized = np.pi * (train_pca - train_pca.min()) / (train_pca.max() - train_pca.min()) - np.pi/2
    test_normalized = np.pi * (test_pca - train_pca.min()) / (train_pca.max() - train_pca.min()) - np.pi/2
    
    print(f"PCA explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Total explained variance: {pca.explained_variance_ratio_.sum():.4f}")
    
    return torch.tensor(train_normalized, dtype=torch.float64), torch.tensor(test_normalized, dtype=torch.float64)

# 데이터 전처리
train_processed, test_processed = preprocess_data(train_data, test_data)
print(f"Processed train data shape: {train_processed.shape}")
print(f"Processed test data shape: {test_processed.shape}")

PCA explained variance ratio: [0.29666528 0.11324686 0.06021043 0.03376736 0.03121631 0.02388517
 0.02002727 0.01644408]
Total explained variance: 0.5955
Processed train data shape: torch.Size([12000, 8])
Processed test data shape: torch.Size([2000, 8])


In [4]:
# 양자 회로 정의
n_qubits = 8
n_layers = 3  # 회로 깊이 제한을 고려

# 양자 디바이스 설정
dev = qml.device('default.qubit', wires=n_qubits)

def angle_encoding(inputs, wires):
    """Angle encoding for input data"""
    for i, wire in enumerate(wires):
        qml.RY(inputs[i], wires=wire)

def variational_layer(params, wires):
    """Variational layer with parameterized gates"""
    # Single qubit rotations
    for i, wire in enumerate(wires):
        qml.RY(params[i], wires=wire)
    
    # Entangling gates (circular connectivity)
    for i in range(len(wires)):
        qml.CNOT(wires=[wires[i], wires[(i + 1) % len(wires)]])

@qml.qnode(dev)
def quantum_circuit(inputs, params):
    """Complete quantum circuit"""
    wires = range(n_qubits)
    
    # Data encoding
    angle_encoding(inputs, wires)
    
    # Variational layers
    for layer in range(n_layers):
        layer_params = params[layer * n_qubits:(layer + 1) * n_qubits]
        variational_layer(layer_params, wires)
    
    # Measurement
    return qml.expval(qml.PauliZ(0))

# 파라미터 개수 확인
total_params = n_layers * n_qubits
print(f"Total trainable parameters: {total_params}")
print(f"Circuit depth estimate: ~{n_layers * 2 + 1}")

# 회로 정보 출력
dummy_input = torch.zeros(n_qubits)
dummy_params = torch.zeros(total_params)
print("\nQuantum circuit created successfully!")
print(qml.draw(quantum_circuit)(dummy_input, dummy_params))

Total trainable parameters: 24
Circuit depth estimate: ~7

Quantum circuit created successfully!
0: ──RY(0.00)──RY(0.00)─╭●───────────────────╭X──RY(0.00)─╭●───────────────────╭X──RY(0.00)─╭● ···
1: ──RY(0.00)──RY(0.00)─╰X─╭●────────────────│───RY(0.00)─╰X─╭●────────────────│───RY(0.00)─╰X ···
2: ──RY(0.00)──RY(0.00)────╰X─╭●─────────────│───RY(0.00)────╰X─╭●─────────────│───RY(0.00)─── ···
3: ──RY(0.00)──RY(0.00)───────╰X─╭●──────────│───RY(0.00)───────╰X─╭●──────────│───RY(0.00)─── ···
4: ──RY(0.00)──RY(0.00)──────────╰X─╭●───────│───RY(0.00)──────────╰X─╭●───────│───RY(0.00)─── ···
5: ──RY(0.00)──RY(0.00)─────────────╰X─╭●────│───RY(0.00)─────────────╰X─╭●────│───RY(0.00)─── ···
6: ──RY(0.00)──RY(0.00)────────────────╰X─╭●─│───RY(0.00)────────────────╰X─╭●─│───RY(0.00)─── ···
7: ──RY(0.00)──RY(0.00)───────────────────╰X─╰●──RY(0.00)───────────────────╰X─╰●──RY(0.00)─── ···

0: ··· ───────────────────╭X─┤  <Z>
1: ··· ─╭●────────────────│──┤     
2: ··· ─╰X─╭●─────────────│──┤     
3:

In [5]:
# PyTorch 양자 모델 래퍼
class QuantumClassifier(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        
        # 학습 가능한 파라미터
        self.params = nn.Parameter(torch.randn(n_layers * n_qubits) * 0.1)
        
    def forward(self, x):
        batch_size = x.shape[0]
        outputs = []
        
        for i in range(batch_size):
            output = quantum_circuit(x[i], self.params)
            outputs.append(output)
        
        outputs = torch.stack(outputs)
        # 시그모이드를 적용하여 확률로 변환
        return torch.sigmoid(outputs)

# 모델 초기화
model = QuantumClassifier(n_qubits, n_layers)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

Model parameters: 24


In [6]:
# 훈련 설정
batch_size = 32
learning_rate = 0.01
epochs = 50

# 데이터 로더 생성
train_dataset = TensorDataset(train_processed, train_labels.float())
test_dataset = TensorDataset(test_processed, test_labels.float())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 손실 함수와 옵티마이저
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 훈련 함수
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc="Training")):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        predicted = (output > 0.5).double()
        total += target.size(0)
        correct += (predicted == target).sum().item()
    
    return total_loss / len(train_loader), correct / total

# 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            
            predicted = (output > 0.5).double()
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    return total_loss / len(test_loader), correct / total

print("Training setup complete!")

Training setup complete!


In [7]:
# 타입 호환성 문제 해결
torch.set_default_dtype(torch.float64)

# 데이터 로더 재생성 (double precision)
train_dataset = TensorDataset(train_processed, train_labels.double())
test_dataset = TensorDataset(test_processed, test_labels.double())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# QuantumClassifier 클래스 정의
class QuantumClassifierFixed(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        
        # 학습 가능한 파라미터 (double precision)
        self.params = nn.Parameter(torch.randn(n_layers * n_qubits, dtype=torch.float64) * 0.1)
        
    def forward(self, x):
        batch_size = x.shape[0]
        outputs = []
        
        for i in range(batch_size):
            output = quantum_circuit(x[i], self.params)
            outputs.append(output)
        
        outputs = torch.stack(outputs)
        # 시그모이드를 적용하여 확률로 변환 (이미 double precision)
        return torch.sigmoid(outputs)

# 수정된 모델로 재초기화
model = QuantumClassifierFixed(n_qubits, n_layers)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

# 훈련 실행
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

print("Starting training...")
for epoch in range(epochs):
    # 훈련
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    
    # 평가
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    
    # 기록
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    print(f"Epoch {epoch+1}/{epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    print()

print("Training completed!")

Model parameters: 24
Starting training...


Training: 100%|██████████| 375/375 [02:14<00:00,  2.78it/s]


Epoch 1/50:
  Train Loss: 0.6419, Train Acc: 0.7501
  Test Loss: 0.6324, Test Acc: 0.7750



Training: 100%|██████████| 375/375 [02:13<00:00,  2.80it/s]


Epoch 2/50:
  Train Loss: 0.6306, Train Acc: 0.7761
  Test Loss: 0.6319, Test Acc: 0.7735



...

Training: 100%|██████████| 375/375 [02:27<00:00,  2.54it/s]


Epoch 18/50:
  Train Loss: 0.6309, Train Acc: 0.7827
  Test Loss: 0.6323, Test Acc: 0.7585



Training: 100%|██████████| 375/375 [02:27<00:00,  2.54it/s]


Epoch 19/50:
  Train Loss: 0.6306, Train Acc: 0.7789
  Test Loss: 0.6317, Test Acc: 0.7750



Training:  47%|████▋     | 177/375 [01:09<01:16,  2.60it/s]

In [ ]:
# 결과 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 손실 그래프
ax1.plot(train_losses, label='Train Loss', color='blue')
ax1.plot(test_losses, label='Test Loss', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True)

# 정확도 그래프
ax2.plot(train_accuracies, label='Train Accuracy', color='blue')
ax2.plot(test_accuracies, label='Test Accuracy', color='red')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Test Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# 최종 결과 출력
print(f"Final Test Accuracy: {test_accuracies[-1]:.4f}")
print(f"Best Test Accuracy: {max(test_accuracies):.4f}")

In [ ]:
# 모델 요약 및 제약조건 확인
print("=== 모델 요약 ===")
print(f"사용된 큐빗 수: {n_qubits}")
print(f"회로 레이어 수: {n_layers}")
print(f"총 학습 가능한 파라미터 수: {sum(p.numel() for p in model.parameters())}")
print(f"예상 회로 깊이: ~{n_layers * 2 + 1}")

print("\n=== 제약조건 확인 ===")
print(f"큐빗 수 제한 (≤8): {n_qubits <= 8} ✓" if n_qubits <= 8 else f"큐빗 수 제한 (≤8): {n_qubits <= 8} ✗")
print(f"회로 깊이 제한 (≤30): {n_layers * 2 + 1 <= 30} ✓" if n_layers * 2 + 1 <= 30 else f"회로 깊이 제한 (≤30): {n_layers * 2 + 1 <= 30} ✗")
print(f"파라미터 수 제한 (≤60): {sum(p.numel() for p in model.parameters()) <= 60} ✓" if sum(p.numel() for p in model.parameters()) <= 60 else f"파라미터 수 제한 (≤60): {sum(p.numel() for p in model.parameters()) <= 60} ✗")

print("\n=== 사용된 기법 ===")
print("- 데이터 인코딩: Angle Encoding")
print("- 차원 축소: PCA (784 → 8 차원)")
print("- 양자 회로: Variational Quantum Circuit")
print("- 얽힘 구조: Circular CNOT connectivity")
print("- 측정: Pauli-Z expectation value")